# Running Back Analysis

In [1]:
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
DATABASE = "../database.db"
con = duckdb.connect(DATABASE, read_only=True)

In [3]:
query = """
SELECT
    season,
    week,
    category,
    player_id,
    player_name,
    position,
    team,
    opponent,
    ppr_scoring_fantasy_points,
    rushing_attempts,
    rushing_yards,
    rushing_touchdowns,
    receiving_targets,
    receiving_yards,
    receiving_touchdowns
FROM player_week_data
WHERE
    category = 'stat'
    AND season = 2023
    AND position = 'RB'
LIMIT 10
"""
con.execute(query).fetchdf()

,season,week,category,player_id,player_name,position,team,opponent,ppr_scoring_fantasy_points,rushing_attempts,rushing_yards,rushing_touchdowns,receiving_targets,receiving_yards,receiving_touchdowns
0,2023,1,stat,10216,Kenny McIntosh,RB,SEA,LAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,1,stat,10219,Chris Rodriguez,RB,WAS,ARI,0.7,3.0,7.0,0.0,0.0,0.0,0.0
2,2023,1,stat,10223,Eric Gray,RB,NYG,DAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023,1,stat,10235,Roschon Johnson,RB,CHI,GB,17.5,5.0,20.0,1.0,7.0,35.0,0.0
4,2023,1,stat,1034,Brandon Bolden,RB,LV,DEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2023,1,stat,10861,Xazavian Valladay,RB,NYJ,BUF,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2023,1,stat,10870,Deneric Prince,RB,KC,DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2023,1,stat,11060,Robert Burns,RB,CHI,GB,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2023,1,stat,11104,SaRodorick Thompson,RB,SEA,LAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2023,1,stat,11114,Elijah Dotson,RB,LAC,MIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
query = """
CREATE OR REPLACE TEMP VIEW rusher_week_share_view AS
WITH rusher_week_data AS (
    SELECT
        *,
        IF(rushing_yards < 0, 0, rushing_yards) AS non_zero_rushing_yards,
        IF(rushing_attempts > 0, rushing_yards / rushing_attempts, null) AS rushing_yards_per_attempt
    FROM player_week_data
    WHERE
        category = 'stat'
        AND season = 2023
        AND position = 'RB'
        AND rushing_attempts > 0
),

team_week_rushing_data AS (
    SELECT
        season,
        week,
        team,
        SUM(rushing_attempts) AS team_rushing_attempts,
        SUM(non_zero_rushing_yards) AS team_rushing_yards,
        ARGMAX(player_id, rushing_attempts) AS top_rusher_by_attempts,
        ARGMAX(player_id, rushing_yards) AS top_rusher_by_yards,
        ARGMAX(player_id, ppr_scoring_fantasy_points) AS top_rusher_by_fantasy_points,
        MAX(rushing_yards_per_attempt) AS top_rushing_yards_per_attempt,
        MAX(rushing_attempts) AS top_rush_attempts,
        MAX(non_zero_rushing_yards) AS top_rush_yards,
        MAX(ppr_scoring_fantasy_points) AS top_fantasy_points
    FROM rusher_week_data
    GROUP BY
        season,
        week,
        team
),

rusher_week_share_data AS (
    SELECT
        r.*,
        r.rushing_attempts / t.team_rushing_attempts AS fraction_of_rush_attempts,
        r.non_zero_rushing_yards / t.team_rushing_yards AS fraction_of_rush_yards,
        r.player_id = t.top_rusher_by_attempts AS is_top_rusher_by_attempts,
        r.player_id = t.top_rusher_by_yards AS is_top_rusher_by_yards,
        t.* EXCLUDE (season, week, team)
    FROM rusher_week_data r
    LEFT JOIN team_week_rushing_data t
        ON r.season = t.season
        AND r.week = t.week
        AND r.team = t.team
    ORDER BY
        r.season,
        r.week,
        r.team,
        fraction_of_rush_attempts DESC
)

SELECT *
FROM rusher_week_share_data
"""
con.sql(query)

In [5]:
query = """
SELECT
    team,
    opponent,
    player_id,
    player_name,
    ppr_scoring_fantasy_points
FROM player_week_data
WHERE
    category = 'stat'
    AND season = 2023
    AND week = 7
LIMIT 5
"""
con.execute(query).fetchdf()

,team,opponent,player_id,player_name,ppr_scoring_fantasy_points
0,NO,JAX,11105,Shaquan Davis,0.0
1,JAX,NO,11139,Elijah Cooks,0.0
2,NO,JAX,3199,Michael Thomas,13.2
3,JAX,NO,4198,Jamal Agnew,7.5
4,JAX,NO,4950,Christian Kirk,19.0


In [6]:
query = """
CREATE OR REPLACE TEMP VIEW rusher_season_total_view AS
SELECT
    team,
    player_id,
    MAX(player_name) AS player_name,
    COUNT(DISTINCT game_id) AS total_active_games,
    SUM(ppr_scoring_fantasy_points) AS total_fantasy_points
FROM player_week_data
WHERE
    category = 'stat'
    AND season = 2023
    AND position = 'RB'
    AND active_games > 0
GROUP BY
    team,
    player_id
ORDER BY
    total_active_games DESC,
    total_fantasy_points DESC
"""
con.sql(query)

query = """
SELECT *
FROM rusher_season_total_view
LIMIT 5
"""
con.execute(query).fetchdf()

,team,player_id,player_name,total_active_games,total_fantasy_points
0,JAX,7543,Travis Etienne,7,138.0
1,NO,4035,Alvin Kamara,7,86.8
2,JAX,9225,Tank Bigsby,7,16.1
3,NO,4149,Jamaal Williams,7,11.5
4,JAX,6694,D'Ernest Johnson,7,7.2


In [13]:
query = """
CREATE OR REPLACE TEMP VIEW player_roster_view AS
WITH league_player AS (
    SELECT
        owner_id,
        UNNEST(players) AS player_id,
        'rostered' AS roster_status
    FROM league_roster
),
league_starter AS (
    SELECT
        owner_id,
        UNNEST(starters) AS player_id,
        'starting' AS roster_status
    FROM league_roster
)

SELECT
    p.owner_id,
    u.display_name AS owner_name,
    JSON_EXTRACT(u.metadata, 'team_name') AS team_name,
    p.player_id,
    COALESCE(s.roster_status, p.roster_status) AS roster_status
FROM league_player p
LEFT JOIN league_starter s
    ON p.owner_id = s.owner_id
    AND p.player_id = s.player_id
LEFT JOIN league_user u
    ON p.owner_id = u.user_id
"""
con.sql(query)

query = """
SELECT *
FROM player_roster_view
LIMIT 5
"""
con.execute(query).fetchdf()

,owner_id,owner_name,team_name,player_id,roster_status
0,456883014945730560,datten,"""Stake Sandwich""",1373,starting
1,456883014945730560,datten,"""Stake Sandwich""",3199,starting
2,456883014945730560,datten,"""Stake Sandwich""",4037,starting
3,456883014945730560,datten,"""Stake Sandwich""",4066,starting
4,456883014945730560,datten,"""Stake Sandwich""",4866,starting


In [61]:
query = """
WITH outperformed_lead_rusher AS (
    SELECT *
    FROM rusher_week_share_view
    WHERE
        top_rusher_by_attempts != top_rusher_by_fantasy_points
        AND player_id = top_rusher_by_fantasy_points
),

outperformed_lead_rusher_season_stat AS (
    SELECT
        team,
        player_id,
        MAX(player_name) AS player_name,
        COUNT(DISTINCT game_id) AS games_beat_lead_rusher,
        SUM(ppr_scoring_fantasy_points)  AS total_points_when_beat_lead_rusher,
        CAST(SUM(IF(ppr_scoring_fantasy_points >= 10, 1, 0)) AS INT) AS games_scored_10_plus
    FROM outperformed_lead_rusher
    GROUP BY
        team,
        player_id
    HAVING games_scored_10_plus > 0
    ORDER BY
        games_scored_10_plus DESC,
        games_beat_lead_rusher DESC,
        total_points_when_beat_lead_rusher DESC
)

SELECT
    r.team,
    r.player_name,
    r.games_scored_10_plus,
    r.games_beat_lead_rusher,
    s.total_active_games,
    ROUND(s.total_fantasy_points / s.total_active_games, 1) AS average_points_per_game,
    COALESCE(p.roster_status, 'available') AS roster_status,
    p.owner_name
FROM outperformed_lead_rusher_season_stat r
LEFT JOIN rusher_season_total_view s
    ON r.team = s.team
    AND r.player_id = s.player_id
LEFT JOIN player_roster_view p
    ON r.player_id = p.player_id
"""
df = con.execute(query).fetchdf()
df.style.apply(lambda x: ["background: lime" if v in {"available", "vingkan"} else "" for v in x], axis=1)

,team,player_name,games_scored_10_plus,games_beat_lead_rusher,total_active_games,average_points_per_game,roster_status,owner_name
0,PIT,Jaylen Warren,3,5,5,10.000000,rostered,edwardjun
1,BAL,Justice Hill,2,3,5,8.700000,rostered,arjitj2
2,ATL,Bijan Robinson,2,2,6,15.800000,starting,TeamSatter
3,TEN,Tyjae Spears,1,2,6,8.200000,rostered,vingkan
4,GB,Aaron Jones,1,1,2,14.600000,starting,vingkan
5,IND,Zack Moss,1,1,5,20.700000,starting,aqures22
6,CHI,Roschon Johnson,1,1,5,7.000000,rostered,edwardjun
7,CLE,Kareem Hunt,1,1,3,7.600000,starting,TeamSatter
8,DET,Jahmyr Gibbs,1,1,4,9.700000,starting,BrotherEzekiel
9,BAL,Gus Edwards,1,1,6,6.700000,rostered,aqures22


In [69]:
query = """
WITH rusher_season_data AS (
SELECT
    team,
    player_id,
    MAX(player_name) AS player_namme,
    SUM(active_games) AS total_active_games,
    SUM(rushing_attempts) AS total_rushing_attempts,
    total_rushing_attempts / total_active_games AS average_attempts_per_game
FROM player_week_data
WHERE
    category = 'stat'
    AND season = 2023
    AND position = 'RB'
    AND active_games > 0
GROUP BY
    team,
    player_id
ORDER BY
    average_attempts_per_game DESC
)

SELECT
    r.*,
    COALESCE(p.roster_status, 'available') AS roster_status,
    p.owner_name,
FROM rusher_season_data r
LEFT JOIN player_roster_view p
    ON r.player_id = p.player_id
LIMIT 50
"""
df = con.execute(query).fetchdf()
df.style.apply(lambda x: ["background: lime" if v in {"available", "vingkan"} else "" for v in x], axis=1)

,team,player_id,player_namme,total_active_games,total_rushing_attempts,average_attempts_per_game,roster_status,owner_name
0,IND,6845,Zack Moss,5.000000,96.000000,19.200000,starting,aqures22
1,DET,5892,David Montgomery,5.000000,94.000000,18.800000,rostered,edwardjun
2,SF,4034,Christian McCaffrey,6.000000,110.000000,18.333333,rostered,edwardjun
3,JAX,7543,Travis Etienne,7.000000,127.000000,18.142857,starting,edwardjun
4,LV,5850,Josh Jacobs,6.000000,107.000000,17.833333,starting,sazonhason
5,NYG,4866,Saquon Barkley,3.000000,53.000000,17.666667,starting,datten
6,SEA,8151,Kenneth Walker,5.000000,83.000000,16.600000,starting,sazonhason
7,TEN,3198,Derrick Henry,6.000000,98.000000,16.333333,rostered,LiveMas
8,HOU,8129,Dameon Pierce,6.000000,97.000000,16.166667,rostered,edwardjun
9,CIN,4018,Joe Mixon,6.000000,96.000000,16.000000,rostered,aqures22
